# Boston Housing Predictor - Phase 2: Advanced Feature Engineering

## Overview
Phase 2 focuses on improving generalization under temporal drift while strictly maintaining
the chronological split (first 70% train, last 30% test). We:

- Use stability analysis to select stable features
- Apply train-quantile winsorization and monotonic transforms
- Engineer ratio/difference interactions among stable features
- Calibrate predictions using train-only bias/trend models
- Tune SVR/GB with time-aware CV on the train segment


In [45]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Ridge, ElasticNet
from sklearn.model_selection import ParameterGrid
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)
print('Libraries imported')


Libraries imported


## 1. Data Loading and Strict Constraint


In [46]:
columns = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT','MEDV']
data = pd.read_csv('data/housing.csv', names=columns, delim_whitespace=True)
censored = (data['MEDV'] >= 50.0).sum()
data = data[data['MEDV'] < 50.0].copy()
X = data.drop('MEDV', axis=1)
y = data['MEDV']
print(f'Data: {X.shape}, censored removed: {censored}')

def strict_chronological_split(X, y, train_size=0.7):
    split = int(len(X)*train_size)
    return X.iloc[:split], X.iloc[split:], y.iloc[:split], y.iloc[split:]

X_train, X_test, y_train, y_test = strict_chronological_split(X, y)
print(f'Train: {X_train.shape[0]} | Test: {X_test.shape[0]} (strict)')


Data: (490, 13), censored removed: 16
Train: 343 | Test: 147 (strict)


## 2. Stability Analysis and Stable Feature Selection


In [47]:
def analyze_feature_stability(X, n_splits=10):
    n = len(X)
    size = n // n_splits
    out = {}
    for col in X.columns:
        if col == 'CHAS':
            continue
        means, stds = [], []
        for i in range(n_splits):
            s = i*size
            e = s+size if i < n_splits-1 else n
            seg = X.iloc[s:e][col]
            means.append(seg.mean())
            stds.append(seg.std())
        mv = np.var(means)
        sv = np.var(stds)
        ov = X[col].var()
        r_mv = mv/ov if ov>0 else 0
        r_sv = sv/ov if ov>0 else 0
        out[col] = r_mv + r_sv
    return pd.Series(out).sort_values()

stability = analyze_feature_stability(X)
stable_features = stability.head(8).index.tolist()
print('Stable features:', stable_features)
X_train_s = X_train[stable_features].copy()
X_test_s = X_test[stable_features].copy()


Stable features: ['RM', 'AGE', 'LSTAT', 'ZN', 'DIS', 'NOX', 'INDUS', 'PTRATIO']


## 3. Phase 2 Feature Transforms (Train stats only)
- Winsorization by train quantiles (e.g., 1st–99th)
- Monotonic transforms for skew (log1p)
- Ratio/difference features among stable signals


In [48]:
def winsorize_by_train_quantiles(X_tr, X_te, lower=0.01, upper=0.99):
    Xtr = X_tr.copy()
    Xte = X_te.copy()
    qs = Xtr.quantile([lower, upper])
    for c in Xtr.columns:
        low, high = qs.loc[lower, c], qs.loc[upper, c]
        Xtr[c] = Xtr[c].clip(lower=low, upper=high)
        Xte[c] = Xte[c].clip(lower=low, upper=high)
    return Xtr, Xte

def monotonic_transforms(X_tr, X_te, log_candidates=None):
    Xtr = X_tr.copy()
    Xte = X_te.copy()
    if log_candidates is None:
        log_candidates = [c for c in Xtr.columns if (Xtr[c] > 0).all() and (Xte[c] > 0).all()]
    for c in log_candidates:
        Xtr[c+'_log1p'] = np.log1p(Xtr[c])
        Xte[c+'_log1p'] = np.log1p(Xte[c])
    return Xtr, Xte

def ratio_diff_features(X_tr, X_te, pairs):
    Xtr = X_tr.copy()
    Xte = X_te.copy()
    for a, b in pairs:
        if a in Xtr.columns and b in Xtr.columns:
            denom_tr = np.where(Xtr[b]==0, 1e-6, Xtr[b])
            denom_te = np.where(Xte[b]==0, 1e-6, Xte[b])
            Xtr[f'{a}_over_{b}'] = Xtr[a] / denom_tr
            Xte[f'{a}_over_{b}'] = Xte[a] / denom_te
            Xtr[f'{a}_minus_{b}'] = Xtr[a] - Xtr[b]
            Xte[f'{a}_minus_{b}'] = Xte[a] - Xte[b]
    return Xtr, Xte

# Apply transforms
Xtr_w, Xte_w = winsorize_by_train_quantiles(X_train_s, X_test_s)
Xtr_m, Xte_m = monotonic_transforms(Xtr_w, Xte_w)
pairs = [('LSTAT','RM'), ('NOX','DIS')]
Xtr_f, Xte_f = ratio_diff_features(Xtr_m, Xte_m, pairs)
print('Transformed train/test shapes:', Xtr_f.shape, Xte_f.shape)

# Scale with train-only stats
scaler = RobustScaler()
Xtr = scaler.fit_transform(Xtr_f)
Xte = scaler.transform(Xte_f)
print('Scaled shapes:', Xtr.shape, Xte.shape)


Transformed train/test shapes: (343, 19) (147, 19)
Scaled shapes: (343, 19) (147, 19)


## 4. Time-aware CV on Train and Hyperparameter Tuning
We use forward-style splits within the first 70% to tune models.

In [49]:
def forward_cv_indices(n, k=5):
    # Expanding window CV within train portion
    sizes = np.linspace(0.6, 0.95, k)
    idx = np.arange(n)
    folds = []
    for s in sizes:
        split = int(n*s)
        if split < n-1:
            folds.append((idx[:split], idx[split:]))
    return folds

def tune_model(model_name, Xtr, ytr):
    results = []
    folds = forward_cv_indices(len(Xtr), k=5)
    if model_name == 'SVR':
        grid = ParameterGrid({'C':[0.5,1,3,10], 'gamma':['scale',0.05,0.1,0.2]})
        base = SVR(kernel='rbf')
    elif model_name == 'GB':
        grid = ParameterGrid({'n_estimators':[200,300,500], 'learning_rate':[0.03,0.05], 'max_depth':[2,3]})
        base = GradientBoostingRegressor(random_state=42)
    else:
        return None
    for params in grid:
        rmses = []
        for tr, va in folds:
            model = base.set_params(**params)
            model.fit(Xtr[tr], ytr.iloc[tr])
            pred = model.predict(Xtr[va])
            rmses.append(np.sqrt(mean_squared_error(ytr.iloc[va], pred)))
        results.append((np.mean(rmses), params))
    results.sort(key=lambda x: x[0])
    return results[0] if results else None

best_svr = tune_model('SVR', Xtr, y_train)
best_gb = tune_model('GB', Xtr, y_train)
print('Best SVR:', best_svr)
print('Best GB:', best_gb)


Best SVR: (np.float64(3.3071231688475473), {'C': 10, 'gamma': 0.05})
Best GB: (np.float64(3.0402137080134883), {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 200})


## 5. Train Final Models and Target Calibration (train-only)
We fit models on full train, then learn a bias/trend calibrator on train residuals and apply to test predictions.

In [50]:
def train_final(model_name, best, Xtr, ytr):
    if model_name=='SVR':
        model = SVR(kernel='rbf', **best[1]) if best else SVR(kernel='rbf')
    elif model_name=='GB':
        params = best[1] if best else {'n_estimators':300,'learning_rate':0.03,'max_depth':3}
        model = GradientBoostingRegressor(random_state=42, **params)
    elif model_name=='Ridge':
        model = Ridge(alpha=1.0)
    else:
        model = ElasticNet(alpha=0.001, l1_ratio=0.5)
    model.fit(Xtr, ytr)
    return model

svr = train_final('SVR', best_svr, Xtr, y_train)
gb = train_final('GB', best_gb, Xtr, y_train)
ridge = train_final('Ridge', None, Xtr, y_train)
enet = train_final('EN', None, Xtr, y_train)

def fit_calibrator(y_true, y_pred):
    # Simple linear calibrator on train residual trend: y = a*y_pred + b
    Xc = np.vstack([y_pred, np.ones_like(y_pred)]).T
    a, b = np.linalg.lstsq(Xc, y_true, rcond=None)[0]
    return a, b

def apply_calibrator(y_pred, a, b):
    return a*y_pred + b

models = {'SVR': svr, 'GB': gb, 'Ridge': ridge, 'ElasticNet': enet}
results = []
for name, m in models.items():
    # Train predictions for calibration
    pred_tr = m.predict(Xtr)
    a, b = fit_calibrator(y_train.values, pred_tr)
    # Test predictions (before/after calibration)
    pred_te = m.predict(Xte)
    pred_te_cal = apply_calibrator(pred_te, a, b)
    # Metrics
    rmse_raw = np.sqrt(mean_squared_error(y_test, pred_te))
    r2_raw = r2_score(y_test, pred_te)
    rmse_cal = np.sqrt(mean_squared_error(y_test, pred_te_cal))
    r2_cal = r2_score(y_test, pred_te_cal)
    results.append({'model':name,'rmse_raw':rmse_raw,'r2_raw':r2_raw,'rmse_cal':rmse_cal,'r2_cal':r2_cal})
df_res = pd.DataFrame(results)
print(df_res)


        model  rmse_raw    r2_raw  rmse_cal    r2_cal
0         SVR  3.814822  0.506698  3.754015  0.522299
1          GB  6.000421 -0.220472  5.954715 -0.201950
2       Ridge  5.903238 -0.181259  5.880982 -0.172368
3  ElasticNet  5.558285 -0.047240  5.542944 -0.041467


## 6. Summary
We report raw and calibrated metrics on the strict test set and compare improvements.